## Errors
This simple notebook can be used list errors that happened while running various studies.

In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core import ultratb

ultratb.VerboseTB.tb_highlight = "bg:#3e0054"

In [ ]:
import os

if not os.getcwd().endswith("syftr"):
    os.chdir(os.path.dirname(os.getcwd()))
    print(f"Changed working directory to: {os.getcwd()}")

In [ ]:
import pandas as pd

from syftr.optuna_helper import get_study_names

INCLUDE_REGEX = [
    # ".*llm2--thinking.*",
    ".*llm2--test.*",
]
EXCLUDE_REGEX = []

study_names = get_study_names(
    include_regex=INCLUDE_REGEX,
    exclude_regex=EXCLUDE_REGEX,
)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
from syftr.optuna_helper import get_failed_trials

bad_trials = get_failed_trials(study_names=study_names)
bad_trials

In [ ]:
df_grouped = bad_trials[[
    "user_attrs_metric_exception_class", 
    "user_attrs_metric_exception_message", 
    "user_attrs_metric_exception_stacktrace", 
    "user_attrs_metric_failed"
]].groupby(
    by=["user_attrs_metric_exception_class", "user_attrs_metric_exception_message"]
).agg(
    user_attrs_metric_exception_stacktrace=('user_attrs_metric_exception_stacktrace', 'first'),
    user_attrs_metric_failed_count=('user_attrs_metric_failed', 'count')
).reset_index()

df_grouped = df_grouped.sort_values(by="user_attrs_metric_failed_count", ascending=False)
df_grouped = df_grouped[df_grouped["user_attrs_metric_exception_class"] != "TrialPruned"]
df_grouped

In [ ]:
for _, row in df_grouped.iterrows():
    print("#" * 50)
    print(f"Number of failures for {row['user_attrs_metric_exception_class']}: {row['user_attrs_metric_failed_count']}")
    print(f"Example stacktrace: {row['user_attrs_metric_exception_stacktrace']}")